<a href="https://colab.research.google.com/github/DivyaBharathi30/iris-recognition/blob/main/iris_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.1.30 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/107.7 GB disk)


In [ ]:
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg'

Ultralytics YOLOv8.1.30 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Found https://ultralytics.com/images/zidane.jpg locally at zidane.jpg
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 179.2ms
Speed: 5.9ms preprocess, 179.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict6
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:10<00:00, 76.4MB/s]


In [ ]:
# Validate YOLOv8n on COCO8 val
!yolo val model=yolov8n.pt data=coco8.yaml

/bin/bash: line 1: yolo: command not found


In [1]:
import cv2
import numpy as np

In [7]:
image1 = cv2.imread("iris_image.jpg")
image2 = cv2.imread("iris_image_1.jpg")

In [3]:
pip install opencv-python

In [4]:
pip install opencv-contrib-python

In [8]:
print(image1)

[[[ 22  22  22]
  [ 24  24  24]
  [ 28  28  28]
  ...
  [ 55  55  55]
  [ 53  53  53]
  [ 53  53  53]]

 [[ 27  27  27]
  [ 29  29  29]
  [ 32  32  32]
  ...
  [ 53  53  53]
  [ 51  51  51]
  [ 51  51  51]]

 [[ 35  35  35]
  [ 36  36  36]
  [ 38  38  38]
  ...
  [ 54  54  54]
  [ 53  53  53]
  [ 53  53  53]]

 ...

 [[ 81  81  81]
  [ 90  90  90]
  [ 94  94  94]
  ...
  [ 66  66  66]
  [ 48  48  48]
  [ 73  73  73]]

 [[ 71  71  71]
  [ 83  83  83]
  [ 91  91  91]
  ...
  [ 66  66  66]
  [ 88  88  88]
  [127 127 127]]

 [[ 70  70  70]
  [ 91  91  91]
  [ 93  93  93]
  ...
  [ 69  69  69]
  [ 64  64  64]
  [161 161 161]]]


In [9]:
print(image2)

[[[ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]
  ...
  [50 50 50]
  [49 49 49]
  [47 47 47]]

 [[10 10 10]
  [10 10 10]
  [12 12 12]
  ...
  [52 52 52]
  [50 50 50]
  [49 49 49]]

 [[13 13 13]
  [13 13 13]
  [14 14 14]
  ...
  [52 52 52]
  [50 50 50]
  [48 48 48]]

 ...

 [[45 45 45]
  [45 45 45]
  [45 45 45]
  ...
  [52 52 52]
  [51 51 51]
  [50 50 50]]

 [[45 45 45]
  [45 45 45]
  [45 45 45]
  ...
  [52 52 52]
  [51 51 51]
  [50 50 50]]

 [[46 46 46]
  [45 45 45]
  [45 45 45]
  ...
  [51 51 51]
  [51 51 51]
  [50 50 50]]]


In [10]:
import cv2
import numpy as np

# Load the reference iris image (authorized)
reference_image = cv2.imread('iris_image.jpg', cv2.IMREAD_GRAYSCALE)

# Load the probe iris image (to be verified)
probe_image = cv2.imread('iris_image.jpg', cv2.IMREAD_GRAYSCALE)

# Preprocess the images (e.g., resize, normalize, enhance)
def preprocess_image(image):
    # Example: Resize the image to a standard size
    resized_image = cv2.resize(image, (100, 100))
    # You can add more preprocessing steps such as normalization or enhancement
    return resized_image

reference_image_processed = preprocess_image(reference_image)
probe_image_processed = preprocess_image(probe_image)


# Extract features from the images
def extract_features(image):
    # Example: Perform circular Hough transform to detect iris boundary
    circles = cv2.HoughCircles(image, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                               param1=100, param2=30, minRadius=10, maxRadius=80)

    if circles is not None:
        # Sort circles by radius
        circles = np.uint16(np.around(circles))
        circles = sorted(circles[0], key=lambda x: x[2], reverse=True)

        # Extract iris region using the largest circle
        x, y, r = circles[0]
        iris_region = image[y - r:y + r, x - r:x + r]

        # Normalize iris region (e.g., contrast enhancement, normalization)
        normalized_iris = cv2.equalizeHist(iris_region)

        # Resize the normalized iris region to a standard size
        normalized_iris = cv2.resize(normalized_iris, (50, 100))

        return normalized_iris.flatten()

    else:
        return None

reference_features = extract_features(reference_image_processed)
probe_features = extract_features(probe_image_processed)

# Match the features (e.g., using cosine similarity)
if reference_features is not None and probe_features is not None:
    similarity_score = np.sum(np.abs(reference_features - probe_features))
    # Set a threshold for similarity score to classify as authorized or unauthorized
    threshold = 500

    # Classify as authorized or unauthorized based on the similarity score
    if similarity_score <= threshold:
        print("Authorized")
    else:
        print("Unauthorized")
else:
    print("Feature extraction failed. Unable to proceed.")


Authorized
